In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest, proportions_chisquare
import statsmodels.api as sm

In [2]:
import sys
from pathlib import Path
import os
cwd = os.getcwd()
parent = str(Path(cwd).parents[0])
sys.path.append(parent)

In [3]:
#importing custom functions for analysis

from lib.functions import ci_calc, z_test, summarizer, check_dupes

#ci_calc will compute a simple confidence interval around a proportion
#summarizer gives a nice output of proprotions and CIs
#get_combos helps to get intersections
#check_dupes helps with the date analysis

# Data Loading and Setup

In [4]:
#Loading in all the data and creating the analysis dataset.

#Primary dataset with dates set
df = pd.read_csv(parent + '/data/final_dataset/final_dataset.csv')
df['euctr_results_date'] = pd.to_datetime(df['euctr_results_date'])
df['ctgov_results_date'] = pd.to_datetime(df['ctgov_results_date'])
df['isrctn_results_date'] = pd.to_datetime(df['isrctn_results_date'])
df['journal_pub_date'] = pd.to_datetime(df['journal_pub_date'])

#Manually collected sponsor data for regression
regression = pd.read_csv(parent + '/data/additional_data/manual_reg_data.csv')

#Regression data derived from the EUCTR
other_reg_data = pd.read_csv(parent + '/data/additional_data/spon_country_data.csv')

#the original sample (and replacements) for data on inferred dates
sample = pd.read_csv(parent + '/data/samples/euctr_search_sample_final.csv')
replacements  = pd.read_csv(parent + '/data/samples/replacement_sample.csv')

#the results section scrape and making one column we need into dates
dec_results = pd.read_csv(parent + '/data/source_data/' + 'euctr_data_quality_results_scrape_dec_2020.csv.zip')
dec_results['first_version_date'] = pd.to_datetime(dec_results.first_version_date)

In [5]:
#Setting search reference dates
search_start_date = pd.to_datetime('2020-12-11')
primary_search_completion_date = pd.to_datetime('2021-07-22')
last_search_any = pd.to_datetime('2023-01-03')

# Analysis Prep

## Detail the exclusions

In [6]:
#Number and reason for exclusions
exclusions = df[df.replaced == 1]
exclusions.replaced_reason.value_counts()

Withdrawn             15
Ongoing                3
No protocol access     2
Name: replaced_reason, dtype: int64

## Setting up the analysis dataset

In [7]:
#For the analysis, we don't want to include these excluded trials so we will make an analyses dataframe moving forward.
analysis_df = df[df.replaced != 1].reset_index(drop=True)

#Quick check the final dataset is the length we expect
assert(len(analysis_df) == 500)

In [8]:
#Here we make binary variable to indicate whether recorded results were published after our cutoff. 
#Per protocol, nothing available after we began searches should be included in the final results.
#Creating binary variables allows us to only correct for this once, rather than have to keep doing it throughout.

#Included EUCTR results
analysis_df['euctr_results_inc'] = np.where((analysis_df.euctr_results == 'Yes') & 
                                            (analysis_df.euctr_results_date <= search_start_date), 1, 0)

#Included clinicaltrials.gov results
analysis_df['ctgov_results_inc'] = np.where((analysis_df.ctgov_results == 'Yes') & 
                                            (analysis_df.ctgov_results_date <= search_start_date), 1, 0)

#Included ISRCTN results
analysis_df['isrctn_results_inc'] = np.where((analysis_df.isrctn_results == 'Yes') & 
                                            (analysis_df.isrctn_results_date <= search_start_date), 1, 0)

#Included journal results
analysis_df['journal_results_inc'] = np.where((analysis_df.journal_result == 'Yes') & 
                                            (analysis_df.journal_pub_date <= search_start_date), 1, 0)

#A catch-all for any result
analysis_df['any_results_inc'] = np.where(((analysis_df.euctr_results_inc == 1) | 
                                          (analysis_df.ctgov_results_inc == 1) | 
                                          (analysis_df.isrctn_results_inc == 1) | 
                                          (analysis_df.journal_results_inc == 1)), 1, 0)

# Main Analysis

## Results on the EUCTR

In [9]:
euctr_results = analysis_df[(analysis_df.euctr_results_inc == 1)]
total_found_euctr = len(euctr_results)
summarizer(total_found_euctr, len(analysis_df))

Total: 500
Results found: 264
Proportion: 52.8%
95% CI: 48.42-57.18


(0.48424184223256195, 0.528, 0.5717581577674381)

In [10]:
#What do these results look like?
euctr_results.euctr_results_format.value_counts()

Tabular                                      116
CSR Synopsis                                  86
ClinicalTrials.gov Results                    24
Journal Article                                8
Tabular and Journal Article                    8
Short Report                                   6
Report                                         6
Tabular and CSR Synopsis                       4
Tabular and Short Report                       4
Notice of termination with low enrollment      1
Tabular and Report                             1
Name: euctr_results_format, dtype: int64

In [11]:
#Lets get percents and CIs for these.
results_types = euctr_results.euctr_results_format.value_counts().to_frame()
results_types['percent'] = round((results_types.euctr_results_format / total_found_euctr) * 100,2)

#Adding CIs (i'm sure there's a more elegant way to do this with .map() or .apply() 
#but it's too much of a weird edge case to bother
ci_lower = []
ci_upper = []
for x in results_types.euctr_results_format.to_list():
    cis = ci_calc(x, total_found_euctr, printer=False)
    ci_lower.append(round(cis[0] * 100,2))
    ci_upper.append(round(cis[2] * 100,2))
    
results_types['ci_lower'] = ci_lower
results_types['ci_upper'] = ci_upper

results_types = results_types.reset_index().rename(columns={'index':'results_type'})
results_types

,results_type,euctr_results_format,percent,ci_lower,ci_upper
0,Tabular,116,43.94,37.95,49.93
1,CSR Synopsis,86,32.58,26.92,38.23
2,ClinicalTrials.gov Results,24,9.09,5.62,12.56
3,Journal Article,8,3.03,0.96,5.10
4,Tabular and Journal Article,8,3.03,0.96,5.10
5,Short Report,6,2.27,0.47,4.07
6,Report,6,2.27,0.47,4.07
7,Tabular and CSR Synopsis,4,1.52,0.04,2.99
8,Tabular and Short Report,4,1.52,0.04,2.99
9,Notice of termination with low enrollment,1,0.38,-0.36,1.12


In [12]:
#Now we want to group like with like to get our final descriptive stats to report

#Total with just tabular results
ci_calc(results_types[results_types.results_type == 'Tabular'].euctr_results_format[0], total_found_euctr)

Proportion: 43.94%
95% CI: 37.95-49.93


(0.37952383252571037, 0.4393939393939394, 0.4992640462621684)

In [13]:
#Total with just a Document
doc_types = ['CSR Synopsis', 
             'ClinicalTrials.gov Results', 
             'Journal Article', 
             'Short Report', 
             'Report', 
             'Notice of termination with low enrollment']
ci_calc(results_types[results_types.results_type.isin(doc_types)].euctr_results_format.sum(), total_found_euctr)

Proportion: 49.62%
95% CI: 43.59-55.65


(0.43589902145456233, 0.4962121212121212, 0.55652522096968)

In [14]:
#Total with both Tabular and Document results
ci_calc(results_types[~results_types.results_type.isin(doc_types + ['Tabular'])].euctr_results_format.sum(), total_found_euctr)

Proportion: 6.44%
95% CI: 3.48-9.4


(0.03478498177391675, 0.06439393939393939, 0.09400289701396203)

## Cross-Registration and results availability on other registries

In [15]:
#First we'll look at cross-registration on ClinicalTrials.gov
ctg_xreg = analysis_df[analysis_df.nct_id.notnull()]
print(f'Out of {len(analysis_df)} trials {len(ctg_xreg)} were cross-registered on ClinicalTrials.gov')
ci_calc(len(ctg_xreg),len(analysis_df))

Out of 500 trials 339 were cross-registered on ClinicalTrials.gov
Proportion: 67.8%
95% CI: 63.7-71.9


(0.6370443472619469, 0.678, 0.7189556527380532)

In [16]:
#CTG x-reg results rate
ctg_results = ctg_xreg[(ctg_xreg.ctgov_results_inc == 1)]
print(f'Of the {len(ctg_xreg)} trials registered on ClinicalTrials.gov, {len(ctg_results)} had results')
ci_calc(len(ctg_results), len(ctg_xreg))

Of the 339 trials registered on ClinicalTrials.gov, 132 had results
Proportion: 38.94%
95% CI: 33.75-44.13


(0.3374732043090307, 0.3893805309734513, 0.4412878576378719)

In [17]:
#Proportion cross-registered on the ISRCTN 
isrctn_xreg = analysis_df[analysis_df.isrctn_id.notnull()]
print(f'Out of {len(analysis_df)} trials {len(isrctn_xreg)} were cross-registered on the ISRCTN')
ci_calc(len(isrctn_xreg), len(analysis_df))

Out of 500 trials 32 were cross-registered on the ISRCTN
Proportion: 6.4%
95% CI: 4.25-8.55


(0.042546459201334624, 0.064, 0.08545354079866538)

In [18]:
#ISRCTN x-reg results rate
isrctn_results = isrctn_xreg[(isrctn_xreg.isrctn_results_inc == 1)]
print(f'Of the {len(isrctn_xreg)} trials registered on the ISRCTN, {len(isrctn_results)} had results')
ci_calc(len(isrctn_results), len(isrctn_xreg))

Of the 32 trials registered on the ISRCTN, 2 had results
Proportion: 6.25%
95% CI: -2.14-14.64


(-0.021370016617978563, 0.0625, 0.14637001661797855)

In [19]:
#How many trials registered on all three registries
triple_reg = len(analysis_df[analysis_df.nct_id.notnull() & analysis_df.isrctn_id.notnull()])
print(f'{triple_reg} registered on all three registires')
ci_calc(triple_reg, len(analysis_df))

9 registered on all three registires
Proportion: 1.8%
95% CI: 0.63-2.97


(0.006346320615359283, 0.018, 0.029653679384640714)

## Results in the literature

In [20]:
#How many had results in the literature
journal_results = analysis_df[(analysis_df.journal_results_inc == 1)]
print(f'Out of {len(analysis_df)} trials on the EUCTR, {len(journal_results)} had results in the literature')
ci_calc(len(journal_results), len(analysis_df))

Out of 500 trials on the EUCTR, 293 had results in the literature
Proportion: 58.6%
95% CI: 54.28-62.92


(0.5428262226253018, 0.586, 0.6291737773746982)

In [21]:
#How did we find journal results?

journal_results.journal_source.value_counts()

ClinicalTrials.gov                                 117
Google Scholar                                      90
PubMed                                              46
EUCTR                                               28
ISRCTN                                               7
CSR on EUCTR and ClinicalTrials.gov                  1
Forward citation from review on Google scholar       1
Forward citation from ISRCTN articles                1
Citation checked located Cochrane review             1
Linked to in another document in Google Scholar      1
Name: journal_source, dtype: int64

## Summarizing results availability

In [22]:
#Getting lists of ids for all results types

all_trial_ids = set(analysis_df.euctr_id.to_list())
euctr_results_ids = set(euctr_results.euctr_id.to_list())
ctg_results_ids = set(ctg_results.euctr_id.to_list())
isrctn_results_ids = set(isrctn_results.euctr_id.to_list())
journal_results_ids = set(journal_results.euctr_id.to_list())

In [23]:
#Overall, inclusive of duplicates, how many results did we locate?
len(euctr_results_ids) + len(ctg_results_ids) + len(isrctn_results_ids) + len(journal_results_ids)

691

In [24]:
#How many had results anywhere?
results_nowhere = all_trial_ids - euctr_results_ids - ctg_results_ids - isrctn_results_ids - journal_results_ids
has_some_result = len(analysis_df) - len(results_nowhere)
#sense check
assert(len(results_nowhere) + has_some_result == 500)

print(f'{has_some_result} of {len(analysis_df)} trials had results somewhere')
ci_calc(has_some_result,len(analysis_df))

381 of 500 trials had results somewhere
Proportion: 76.2%
95% CI: 72.47-79.93


(0.7246718012864269, 0.762, 0.7993281987135731)

In [25]:
#What did enrollment look like for the trials with no results?
#This uses enrollment numbers from the manual data we collected on enrollment

no_results = analysis_df[analysis_df.euctr_id.isin(results_nowhere)].reset_index(drop=True)
no_results[['euctr_id']].merge(regression[['Trial ID', 'Enrollment']], how='left', left_on='euctr_id', right_on='Trial ID').Enrollment.sum()

33744.0

## What results were unique to each dissemination route?

In [26]:
#How many had results on just the EUCTR?
just_euctr = len(euctr_results_ids - ctg_results_ids - isrctn_results_ids - journal_results_ids)
print(f'{just_euctr} trials had results on just the EUCTR')
ci_calc(just_euctr, len(euctr_results))

54 trials had results on just the EUCTR
Proportion: 20.45%
95% CI: 15.59-25.32


(0.15588717957072745, 0.20454545454545456, 0.25320372952018166)

In [27]:
#How many had results on just on ClinicalTrials.gov?
just_ctg = len(ctg_results_ids - euctr_results_ids - isrctn_results_ids - journal_results_ids)
print(f'{just_ctg} trials had results on just ClinicalTrials.gov')
ci_calc(just_ctg,len(ctg_results))

3 trials had results on just ClinicalTrials.gov
Proportion: 2.27%
95% CI: -0.27-4.82


(-0.0026971240190091124, 0.022727272727272728, 0.048151669473554565)

In [28]:
#How many had results on just on the ISRCTN?
just_isrctn = len(isrctn_results_ids - euctr_results_ids - ctg_results_ids - journal_results_ids)
print(f'{just_isrctn} trials had results on just the ISRCTN')

0 trials had results on just the ISRCTN


In [29]:
#How many had results just in the literature?
just_pub = len(journal_results_ids - euctr_results_ids - ctg_results_ids - isrctn_results_ids)
print(f'{just_pub} trials had results only in a journal publication')
ci_calc(just_pub,len(journal_results))

107 trials had results only in a journal publication
Proportion: 36.52%
95% CI: 31.01-42.03


(0.31005580797466115, 0.3651877133105802, 0.42031961864649925)

In [30]:
#How many have no results on the EUCTR but results anywhere else?
not_euctr = analysis_df[(analysis_df.euctr_results_inc == 0) & ((analysis_df.ctgov_results_inc == 1) | 
                                                               (analysis_df.isrctn_results_inc == 1) | 
                                                               (analysis_df.journal_results_inc == 1))]
print(f'{len(not_euctr)} trials without EUCTR results had results somewhere else')
ci_calc(len(not_euctr), len(analysis_df))

117 trials without EUCTR results had results somewhere else
Proportion: 23.4%
95% CI: 19.69-27.11


(0.19688979249855912, 0.234, 0.2711102075014409)

In [31]:
#How many had results nowhere?
print(f'{len(results_nowhere)} trials had no results located')
ci_calc(len(results_nowhere), len(analysis_df))

119 trials had no results located
Proportion: 23.8%
95% CI: 20.07-27.53


(0.2006718012864269, 0.238, 0.27532819871357306)

## Getting data on combinations of results availability

We will visualise these in an upset chart in the paper

In [95]:
upset_plot_data = analysis_df[['euctr_results_inc', 'ctgov_results_inc', 'isrctn_results_inc', 'journal_results_inc']]

upset_plot_data.to_csv(parent + '/data/graphing_data/upset_data.csv')

# Data Quality, Completion Status, and Reporting

For overall population numbers, see the `Data Processing` notebooke

In [35]:
#Making a new DF for this population to investiage results availability by inferred and available completion dates

full_sample = pd.concat([sample,replacements])

analysis_df_2 = analysis_df.merge(full_sample[['eudract_number', 'inferred']], 
                                  how='left', 
                                  left_on='euctr_id', right_on='eudract_number').drop('eudract_number', axis=1)

inferred = analysis_df_2[analysis_df_2.inferred == 1]
print(f'Inferred: {len(inferred)}; {round((len(inferred)/len(analysis_df_2)) * 100, 2)}%')
stated = analysis_df_2[analysis_df_2.inferred == 0]
print(f'Stated: {len(stated)}; {round((len(stated)/len(analysis_df_2)) * 100, 2)}%')

Inferred: 147; 29.4%
Stated: 353; 70.6%


In [36]:
#How many of the inferred ones had results anywhere?
inferred_res_sw = len(inferred[(inferred.euctr_results_inc == 1) | (inferred.ctgov_results_inc == 1) | (inferred.isrctn_results_inc == 1) | (inferred.journal_results_inc == 1)])
print(f'Inferred Dates with any results: {inferred_res_sw}')
print(f'Total inferred dates: {len(inferred)}')
ci_calc(inferred_res_sw, len(inferred))

Inferred Dates with any results: 70
Total inferred dates: 147
Proportion: 47.62%
95% CI: 39.55-55.69


(0.3954531334145431, 0.47619047619047616, 0.5569278189664093)

In [37]:
#How many of the extracted ones had results anywhere?
stated_res_sw = len(stated[(stated.euctr_results_inc == 1) | (stated.ctgov_results_inc == 1) | (stated.isrctn_results_inc == 1) | (stated.journal_results_inc == 1)])
print(f'Extracted with any results: {stated_res_sw}')
print(f'Total extracted dates: {len(stated)}')
ci_calc(stated_res_sw, len(stated))

Extracted with any results: 311
Total extracted dates: 353
Proportion: 88.1%
95% CI: 84.72-91.48


(0.8472445581989536, 0.8810198300283286, 0.9147951018577036)

In [38]:
#a is the number of trials with results
#b is the total number of trials

a = [inferred_res_sw, stated_res_sw]
b = [len(inferred),len(stated)]

stat, pval = proportions_ztest(a, b)
print(pval)

3.516998150004491e-22


In [39]:
#How many of the inferred ones had results somewhere else?
inferred_res_swe = len(inferred[(inferred.ctgov_results_inc == 1) | (inferred.isrctn_results_inc == 1) | (inferred.journal_results_inc == 1)])
print(f'Inferred with results outside the EUCTR: {inferred_res_swe}')
print(f'Total Inferred: {len(inferred)}')
ci_calc(inferred_res_swe, len(inferred))

Inferred with results outside the EUCTR: 70
Total Inferred: 147
Proportion: 47.62%
95% CI: 39.55-55.69


(0.3954531334145431, 0.47619047619047616, 0.5569278189664093)

In [40]:
#How many of the extracted ones had results somewhere else?
stated_res_swe = len(stated[(stated.ctgov_results_inc == 1) | (stated.isrctn_results_inc == 1) | (stated.journal_results_inc == 1)])
print(f'Extracted with results outside the EUCTR: {stated_res_swe}')
print(f'Total Extracted: {len(stated)}')
ci_calc(stated_res_swe, len(stated))

Extracted with results outside the EUCTR: 257
Total Extracted: 353
Proportion: 72.8%
95% CI: 68.16-77.45


(0.681626281828091, 0.7280453257790368, 0.7744643697299826)

In [41]:
a = [inferred_res_swe, stated_res_swe]
b = [len(inferred),len(stated)]

stat, pval = proportions_ztest(a, b)
print(pval)

6.903275707982106e-08


## Now we have to do this for each registry

In [42]:
#EUCTR

#Results posted to EUCTR by date of search - stated
stated_results_euctr = stated[(stated.euctr_results_inc == 1)]
print(f'Extracted with EUCTR results: {len(stated_results_euctr)}')
print(f'Total extracted: {len(stated)}')
ci_calc(len(stated_results_euctr), len(stated))

print('\n')

#Results posted to EUCTR by date of search - inferred
inferred_results_euctr = inferred[(inferred.euctr_results_inc == 1)]
print(f'Inferred with EUCTR results: {len(inferred_results_euctr)}')
print(f'Total inferred: {len(inferred)}')
ci_calc(len(inferred_results_euctr), len(inferred))

Extracted with EUCTR results: 264
Total extracted: 353
Proportion: 74.79%
95% CI: 70.26-79.32


Inferred with EUCTR results: 0
Total inferred: 147
Proportion: 0.0%
95% CI: 0.0-0.0


(0.0, 0.0, 0.0)

In [43]:
a = [len(inferred_results_euctr), len(stated_results_euctr)]
b = [len(inferred),len(stated)]

stat, pval = proportions_ztest(a, b)
print(pval)

1.376627481812372e-52


In [44]:
#CTG extracted dates
stated_ctg = stated[stated.nct_id.notnull()]
stated_ctg_results = stated_ctg[(stated_ctg.ctgov_results_inc == 1)]
print(f'Extracted with CTG results: {len(stated_ctg_results)}')
print(f'Total CTG extracted: {len(stated_ctg)}')
ci_calc(len(stated_ctg_results), len(stated_ctg))

print('\n')

#CTG inferred dates
inferred_ctg = inferred[inferred.nct_id.notnull()]
inferred_ctg_results = inferred_ctg[(inferred_ctg.ctgov_results_inc == 1)]
print(f'Inferred with CTG results: {len(inferred_ctg_results)}')
print(f'Total CTG inferred: {len(inferred_ctg)}')
ci_calc(len(inferred_ctg_results), len(inferred_ctg))

Extracted with CTG results: 130
Total CTG extracted: 270
Proportion: 48.15%
95% CI: 42.19-54.11


Inferred with CTG results: 2
Total CTG inferred: 69
Proportion: 2.9%
95% CI: -1.06-6.86


(-0.010599874223468034, 0.028985507246376812, 0.06857088871622166)

In [45]:
a = [len(inferred_ctg_results), len(stated_ctg_results)]
b = [len(inferred_ctg),len(stated_ctg)]

stat, pval = proportions_ztest(a, b)
print(pval)

6.011656192173383e-12


In [46]:
#isrctn extracted dates
stated_isrctn = stated[stated.isrctn_id.notnull()]
stated_isrctn_results = stated_isrctn[(stated_isrctn.isrctn_results_inc == 1)]
print(f'Extracted with ISRCTN results: {len(stated_isrctn_results)}')
print(f'Total ISRCTN extracted: {len(stated_isrctn)}')
ci_calc(len(stated_isrctn_results), len(stated_isrctn))

print('\n')

#isrctn inferred dates
inferred_isrctn = inferred[inferred.isrctn_id.notnull()]
inferred_isrctn_results = inferred_isrctn[(inferred_isrctn.isrctn_results_inc == 1)]
print(f'Inferred with ISRCTN results:{len(inferred_isrctn_results)}')
print(f'Total ISRCTN extracted: {len(inferred_isrctn)}')
ci_calc(len(inferred_isrctn_results), len(inferred_isrctn))

Extracted with ISRCTN results: 2
Total ISRCTN extracted: 29
Proportion: 6.9%
95% CI: -2.33-16.12


Inferred with ISRCTN results:0
Total ISRCTN extracted: 3
Proportion: 0.0%
95% CI: 0.0-0.0


(0.0, 0.0, 0.0)

In [47]:
a = [len(inferred_isrctn_results), len(stated_isrctn_results)]
b = [len(inferred_isrctn),len(stated_isrctn)]

stat, pval = proportions_ztest(a, b)
print(pval)

0.6385149374549752


In [48]:
#journal extracted dates
stated_journal = stated[(stated.journal_results_inc == 1)]
print(f'Extracted with results in a Journal: {len(stated_journal)}')
print(f'Total Extracted: {len(stated)}')
ci_calc(len(stated_journal), len(stated))

print('\n')

#journal inferred dates
inferred_journal = inferred[(inferred.journal_results_inc == 1)]
print(f'Inferred with results in a Journal: {len(inferred_journal)}')
print(f'Total Inferred: {len(inferred)}')
ci_calc(len(inferred_journal), len(inferred))

Extracted with results in a Journal: 224
Total Extracted: 353
Proportion: 63.46%
95% CI: 58.43-68.48


Inferred with results in a Journal: 69
Total Inferred: 147
Proportion: 46.94%
95% CI: 38.87-55.01


(0.38871035097043183, 0.46938775510204084, 0.5500651592336498)

In [49]:
a = [len(inferred_journal), len(stated_journal)]
b = [len(inferred),len(stated)]

stat, pval = proportions_ztest(a, b)
print(pval)

0.0006348653817690258


# Publication Date Analysis

In [50]:
date_df = analysis_df[['euctr_id', 'euctr_results', 'euctr_results_date', 'nct_id', 'ctgov_results', 'ctgov_results_date', 
             'isrctn_id', 'isrctn_results', 'isrctn_results_date', 'journal_result', 'journal_pub_date']].reset_index(drop=True)

earliest_results_date = dec_results.first_version_date.min()

In [51]:
#Sense checking to make sure there are no duplicate values
just_dates = date_df[['euctr_results_date','ctgov_results_date', 'isrctn_results_date', 'journal_pub_date']].reset_index(drop=True)
just_dates['test'] = just_dates.apply(check_dupes, axis=1)
just_dates.test.value_counts()

#There are no repeat dates so no need to worry about that.

False    500
Name: test, dtype: int64

In [52]:
#Getting rid of results dates from after we started searching
date_df['euctr_results_date'] = np.where(date_df['euctr_results_date'] > search_start_date, pd.NaT, date_df['euctr_results_date'])
date_df['euctr_results_date'] = pd.to_datetime(date_df['euctr_results_date'])

date_df['ctgov_results_date'] = np.where(date_df['ctgov_results_date'] > search_start_date, pd.NaT, date_df['ctgov_results_date'])
date_df['ctgov_results_date'] = pd.to_datetime(date_df['ctgov_results_date'])

date_df['isrctn_results_date'] = np.where(date_df['isrctn_results_date'] > search_start_date, pd.NaT, date_df['isrctn_results_date'])
date_df['isrctn_results_date'] = pd.to_datetime(date_df['isrctn_results_date'])

date_df['journal_pub_date'] = np.where(date_df['journal_pub_date'] > search_start_date, pd.NaT, date_df['journal_pub_date'])
date_df['journal_pub_date'] = pd.to_datetime(date_df['journal_pub_date'])

In [53]:
#Getting the earliest publication date for each trial
date_df['min_date'] = date_df[['euctr_results_date',
                               'ctgov_results_date', 
                               'isrctn_results_date', 
                               'journal_pub_date']].min(axis=1)

In [54]:
#Getting the total number of results available
date_df['results_counts'] = (date_df == 'Yes').T.sum()

## Pre-EUCTR Results Section trials

In [55]:
#Trials that had a result of some kind before the EUCTR results section launched
pre_euctr = date_df[(date_df.min_date < earliest_results_date)].reset_index(drop=True)

print(len(pre_euctr))

135


In [56]:
#How many of these went on the publish a result on the EUCTR

print(len(pre_euctr[pre_euctr.euctr_results == 'Yes']))

ci_calc(95,135)

94
Proportion: 70.37%
95% CI: 62.67-78.07


(0.6266759528652704, 0.7037037037037037, 0.780731454542137)

In [57]:
#Here we can extract where the earliest result was extracted for a given trial

conds = [pre_euctr.euctr_results_date == pre_euctr.min_date, 
         pre_euctr.ctgov_results_date == pre_euctr.min_date, 
         pre_euctr.isrctn_results_date == pre_euctr.min_date, 
         pre_euctr.journal_pub_date == pre_euctr.min_date]

out = ['EUCTR', 'CTgov', 'ISRCTN', 'Journal']

pre_euctr['earliest_results'] = np.select(conds, out, 'No Result')

In [59]:
#Lets now look at the distribution of where trials were first to report prior to the EUCTR
#This has to be limited to only trials that were also cross-registered on ClinicalTrials.gov to compare
#like with like
first_report_pre = pre_euctr[pre_euctr.nct_id.notnull()].earliest_results.value_counts()
first_report_pre

Journal    67
CTgov      31
Name: earliest_results, dtype: int64

In [69]:
#Can use this to get the CIs for those
#Journals
ci_calc(first_report_pre[0],first_report_pre.sum())

Proportion: 68.37%
95% CI: 59.16-77.57


(0.5915998519916292, 0.6836734693877551, 0.7757470867838809)

In [70]:
#CT gov
ci_calc(first_report_pre[1],first_report_pre.sum())

Proportion: 31.63%
95% CI: 22.43-40.84


(0.22425291321611912, 0.3163265306122449, 0.4084001480083707)

## Post-EUCTR Results Section Trials

In [61]:
#Make the sample
post_euctr = date_df[(date_df.min_date >= earliest_results_date)].reset_index(drop=True)

#Trials with any result after the launch of the EUCTR results section
print(len(post_euctr))

247


In [62]:
#How many of these ended up on the EUCTR at all
len(post_euctr[post_euctr.euctr_results == 'Yes'])

172

In [67]:
#And the CI for that
ci_calc(len(post_euctr[post_euctr.euctr_results == 'Yes']),len(post_euctr))

Proportion: 69.64%
95% CI: 63.9-75.37


(0.6390099628541973, 0.6963562753036437, 0.7537025877530902)

In [64]:
#Adding the earliest dissemination route

conds = [post_euctr.euctr_results_date == post_euctr.min_date, 
         post_euctr.ctgov_results_date == post_euctr.min_date, 
         post_euctr.isrctn_results_date == post_euctr.min_date, 
         post_euctr.journal_pub_date == post_euctr.min_date]

out = ['EUCTR', 'CTgov', 'ISRCTN', 'Journal']

post_euctr['earliest_results'] = np.select(conds, out, 'No Result')

In [65]:
first_report_post = post_euctr[post_euctr.nct_id.notnull()].earliest_results.value_counts()
first_report_post

Journal    89
EUCTR      83
CTgov      20
Name: earliest_results, dtype: int64

In [66]:
first_report_post.sum()

192

In [72]:
#Journal CIs
ci_calc(first_report_post[0], first_report_post.sum())

Proportion: 46.35%
95% CI: 39.3-53.41


(0.39300452712164496, 0.4635416666666667, 0.5340788062116884)

In [73]:
#EUCTR CIs
ci_calc(first_report_post[1], first_report_post.sum())

Proportion: 43.23%
95% CI: 36.22-50.24


(0.36221772884531195, 0.4322916666666667, 0.5023656044880214)

In [74]:
#CTG CIs
ci_calc(first_report_post[2], first_report_post.sum())

Proportion: 10.42%
95% CI: 6.1-14.74


(0.060956747926332336, 0.10416666666666667, 0.147376585407001)

In [77]:
#What about trials not on ClinicalTrials.gov.
first_pub_no_ctg = post_euctr[post_euctr.nct_id.isna()].earliest_results.value_counts()
first_pub_no_ctg

Journal    31
EUCTR      23
ISRCTN      1
Name: earliest_results, dtype: int64

In [78]:
#CI for journal
ci_calc(first_pub_no_ctg[0],first_pub_no_ctg.sum())

Proportion: 56.36%
95% CI: 43.26-69.47


(0.4325678106200809, 0.5636363636363636, 0.6947049166526463)

In [79]:
#CI for EUCTR
ci_calc(first_pub_no_ctg[1],first_pub_no_ctg.sum())

Proportion: 41.82%
95% CI: 28.78-54.85


(0.28781983933186, 0.41818181818181815, 0.5485437970317764)

In [80]:
#CI for ISRCTN
ci_calc(first_pub_no_ctg[2],first_pub_no_ctg.sum())

Proportion: 1.82%
95% CI: -1.71-5.35


(-0.017129092394234546, 0.01818181818181818, 0.05349272875787091)

## Data for Start Year Figure

Here we just get the data we would need and export it. Figures are makde in a separate notebook.

In [93]:
graphing_df = analysis_df[['euctr_id', 
                           'euctr_results_inc', 
                           'any_results_inc']].merge(regression[['Trial ID', 
                                                                 'Trial Start Year']], 
                                                    how='left', left_on='euctr_id', right_on='Trial ID').drop('Trial ID', axis=1)

graphing_df.to_csv(parent + '/data/graphing_data/start_year_data.csv')

# Reporting of Trial IDs

In [97]:
trial_id_df = analysis_df[['euctr_id', 'nct_id', 'isrctn_id', 'journal_results_inc', 'journal_reg_numbers']].reset_index(drop=True)

In [101]:
reg_id_df = trial_id_df[trial_id_df.journal_results_inc == 1].journal_reg_numbers.value_counts(dropna=False).to_frame().reset_index()

In [102]:
reg_id_df

,index,journal_reg_numbers
0,ClinicalTrials.gov,157
1,None,58
2,EUCTR/EudraCT,33
3,"EUCTR/EudraCT, ClinicalTrials.gov",27
4,ISRCTN,7
5,"EUCTR/EudraCT, ISRCTN",6
6,Other registration number,3
7,"ClinicalTrials.gov, ISRCTN",2


In [110]:
reg_id_df[reg_id_df['index'].str.contains('ClinicalTrials.gov')].journal_reg_numbers.sum()

186

In [111]:
reg_id_df[reg_id_df['index'].str.contains('EUCTR/EudraCT')].journal_reg_numbers.sum()

66

In [112]:
reg_id_df[reg_id_df['index'].str.contains('ISRCTN')].journal_reg_numbers.sum()

15

In [113]:
186+66+15

267